# Performing data cleaning

https://www.kaggle.com/datasets/mohamedsiika/data-related-jobs-in-us

In [1]:
import pandas as pd
import re
import numpy as np
import sys
import os 
from sklearn.impute import KNNImputer 

In [2]:
pd.set_option('display.max_columns', 35)

In [3]:
#sys.path.insert(0, os.path.abspath('../src'))
#import functions

In [4]:
data = pd.read_csv('../data/raw/Data_related_jobs_glassdoor.csv',encoding='cp1252')
data

,company,job title,location,job description,salary estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,hourly,rating,python_yn,spark_yn,azure_yn,aws_yn,excel_yn,machine_learning_yn,job_simpl,seniority,description_len,company_age
0,Microsoft,Data & Applied Scientist,"Redmond, WA",Microsoft 365 is a key part of the company’s c...,123486,10000+ Employees,Company - Public,Information Technology,Computer Hardware Development,1975.0,$10+ billion (USD),0,4.4,1,0,0,1,0,1,data scientist,junior,359,47.0
1,UT Southwestern Medical Center,Data Scientist or Bioinformatician (remote),Remote,Center Information:\nThe Quantitative Biomedic...,93500,10000+ Employees,Hospital,Healthcare,Health Care Services & Hospitals,1943.0,$1 to $5 billion (USD),0,4.0,1,0,0,0,0,1,data scientist,mid,267,79.0
2,Notion,"Data Scientist, Growth","New York, NY",About Us:\nWe're on a mission to make it possi...,137853,201 to 500 Employees,Company - Private,Information Technology,Enterprise Software & Network Solutions,2016.0,Unknown / Non-Applicable,0,4.9,1,0,0,0,0,0,data scientist,Senior,589,6.0
3,Net2Aspire,Jr. Data Scientist,Remote,? Apply Statistical and Machine Learning metho...,72500,Unknown,Company - Public,NaN,NaN,NaN,Unknown / Non-Applicable,0,NaN,0,0,0,0,0,1,data scientist,junior,132,NaN
4,Ntropy Network,Data Scientist,Remote,"Over the last few decades, technological innov...",155000,1 to 50 Employees,Company - Private,NaN,NaN,NaN,Unknown / Non-Applicable,0,NaN,1,0,0,1,0,0,data scientist,mid,522,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,YouTube,"Staff Software Engineer, Machine Learning, You...","San Bruno, CA",Minimum qualifications:\nBachelor's degree or ...,141704,1001 to 5000 Employees,Subsidiary or Business Segment,Information Technology,Internet & Web Services,2005.0,Unknown / Non-Applicable,0,4.3,0,0,0,0,0,1,machine learning engineer,Senior,498,17.0
2080,Hunter Engineering,Data Science Co-Op,"Bridgeton, MO",Overview:\nDo you have a passion for data scie...,88383,1001 to 5000 Employees,Company - Private,Manufacturing,Machinery Manufacturing,1946.0,$500 million to $1 billion (USD),0,4.4,1,0,0,0,1,1,other,Senior,349,76.0
2081,precision technologies corp,Jr UI/UX Designer Training and Placement,Remote,If you want to start your IT career as a UI/UX...,70600,201 to 500 Employees,Company - Private,Information Technology,Information Technology Support Services,2008.0,$5 to $25 million (USD),1,4.2,1,0,0,1,1,1,other,junior,391,14.0
2082,Argonne National Laboratory,Postdoctoral Appointee - Probabilistic Machine...,"Lemont, IL",The Mathematics and Computer Science Division ...,54291,1001 to 5000 Employees,Government,Management & Consulting,Research & Development,1946.0,Unknown / Non-Applicable,0,4.5,0,0,0,0,0,1,machine learning engineer,Senior,506,76.0


In [5]:
data.columns

Index(['company', 'job title', 'location', 'job description',
       'salary estimate', 'company_size', 'company_type', 'company_sector',
       'company_industry', 'company_founded', 'company_revenue', 'hourly',
       'rating', 'python_yn', 'spark_yn', 'azure_yn', 'aws_yn', 'excel_yn',
       'machine_learning_yn', 'job_simpl', 'seniority', 'description_len',
       'company_age'],
      dtype='object')

In [6]:
column_names = data.columns
cols = []
for col in data.columns:
    cols.append(col.lower().replace(' ', '_'))

data.columns = cols
print(data.columns)

Index(['company', 'job_title', 'location', 'job_description',
       'salary_estimate', 'company_size', 'company_type', 'company_sector',
       'company_industry', 'company_founded', 'company_revenue', 'hourly',
       'rating', 'python_yn', 'spark_yn', 'azure_yn', 'aws_yn', 'excel_yn',
       'machine_learning_yn', 'job_simpl', 'seniority', 'description_len',
       'company_age'],
      dtype='object')


## States

**As we know that the data cleaning for job titles is already performed in the columns 'job_simpl' and 'seniority' let's go with location and create a new columns just with the states these locations belong:**

In [7]:
data['location'].value_counts(dropna=True)

Remote                565
New York, NY          160
Chicago, IL            69
Mountain View, CA      58
Marysville, MI         54
                     ... 
Emeryville, CA          1
Indianapolis, IN        1
Franklin Lakes, NJ      1
Chantilly, VA           1
Redwood City, CA        1
Name: location, Length: 240, dtype: int64

In [8]:
location_counts = data['location'].value_counts(dropna=False, sort=True)

location_counts_df = pd.DataFrame(location_counts)
location_counts_df = location_counts_df.reset_index()
location_counts_df.columns = ['locations', 'frequency'] 
location_counts_df

,locations,frequency
0,Remote,565
1,"New York, NY",160
2,"Chicago, IL",69
3,"Mountain View, CA",58
4,"Marysville, MI",54
...,...,...
235,"Emeryville, CA",1
236,"Indianapolis, IN",1
237,"Franklin Lakes, NJ",1
238,"Chantilly, VA",1


In [9]:
#%%writefile --functions.py
def pattern_lister(row, pattern):
    list_fulfill_pattern=[]
    list_not_fulfill=[]
    empty=[]
    for x in row:
        match=re.findall(pattern, x)
        if match != empty and match not in list_fulfill_pattern:
            list_fulfill_pattern.append(match)
        elif match == empty:
            list_not_fulfill.append(x)
    return list_fulfill_pattern, list_not_fulfill         

In [10]:
states, other_or_typos = pattern_lister(location_counts_df['locations'], pattern='[A-Z][A-Z]')
print(states)
print(other_or_typos)

[['NY'], ['IL'], ['CA'], ['MI'], ['TX'], ['WA'], ['OH'], ['VA'], ['GA'], ['MD'], ['CO'], ['NC'], ['UT'], ['PA'], ['MA'], ['NM'], ['MO'], ['DC'], ['FL'], ['NJ'], ['DE'], ['MN'], ['SC'], ['AZ'], ['WI'], ['OK'], ['NV'], ['OR'], ['LA'], ['TN'], ['IA'], ['ID'], ['CT'], ['AR'], ['PR'], ['IN'], ['AF', 'VA'], ['MS'], ['AL'], ['RI'], ['KS'], ['NE']]
['Remote', 'South Carolina', 'Manhattan', 'United States', 'Connecticut', 'California', 'Texas', 'New York State', 'North Carolina', 'Massachusetts', 'Township of Lawrence', 'Wisconsin', 'Illinois', 'Texhoma', 'Arizona', 'Long Island-Queens']


In [11]:
for x in location_counts_df['locations']:
    pattern='[A-Z][A-Z]'
    match=re.findall(pattern, x)
    if match == ['AF', 'VA']:
        print(x)

Langley AFB, VA


In [12]:
final_list = []
empty=[]
pattern='[A-Z][A-Z]'
for x in data['location']:
    match=re.findall(pattern, x)
    if match in states and match != 'AF':
        final_list.append(match[0])
    elif match in states and match == ['AF', 'VA']:
        final_list.append('VA')
    elif x == 'Remote':
        final_list.append(x)
    elif x in ['Manhattan','New York State', 'Long Island-Queens']:
        final_list.append('NY')
    elif x == 'South Carolina':
        final_list.append('SC')
    elif x == 'Connecticut':
        final_list.append('CT') 
    elif x == 'California':
        final_list.append('CA')  
    elif x == 'Texas':
        final_list.append('TX') 
    elif x == 'North Carolina':
        final_list.append('NC') 
    elif x == 'Massachusetts':
        final_list.append('MA')
    elif x == 'Township of Lawrence':
        final_list.append('NJ') 
    elif x == 'Wisconsin':
        final_list.append('WI') 
    elif x == 'Illinois':
        final_list.append('IL')
    elif x == 'Texhoma':
        final_list.append('OK')
    elif x == 'Arizona':
        final_list.append('AZ')
    else:
        final_list.append('Unknown')    

print(final_list)
        

['WA', 'Remote', 'NY', 'Remote', 'Remote', 'IL', 'Remote', 'GA', 'VA', 'NY', 'NY', 'NY', 'CA', 'Remote', 'MI', 'Remote', 'MA', 'WA', 'PA', 'Remote', 'CA', 'Remote', 'NY', 'CA', 'WI', 'RI', 'MA', 'Remote', 'TX', 'Remote', 'CA', 'MO', 'NY', 'MI', 'Remote', 'WA', 'NY', 'DC', 'WA', 'CA', 'Remote', 'Remote', 'Remote', 'LA', 'MA', 'NY', 'Remote', 'CA', 'Remote', 'PA', 'Remote', 'MD', 'Remote', 'Remote', 'Remote', 'NC', 'VA', 'Remote', 'NJ', 'DC', 'CA', 'CA', 'NJ', 'FL', 'IA', 'MI', 'Remote', 'Remote', 'Remote', 'VA', 'CA', 'NJ', 'Remote', 'CO', 'Remote', 'CA', 'Remote', 'LA', 'MA', 'TX', 'Remote', 'CA', 'Remote', 'CO', 'Remote', 'Remote', 'FL', 'NM', 'Remote', 'Remote', 'Remote', 'CA', 'CA', 'Remote', 'Remote', 'CA', 'MN', 'Remote', 'VA', 'NC', 'IN', 'NY', 'TX', 'Remote', 'CA', 'Remote', 'NY', 'CA', 'NJ', 'Remote', 'WA', 'CA', 'CA', 'Remote', 'Remote', 'IN', 'NJ', 'Remote', 'NY', 'VA', 'CO', 'DC', 'CA', 'CA', 'MN', 'Remote', 'Remote', 'WA', 'Remote', 'Remote', 'Remote', 'NY', 'IL', 'Remote',

In [13]:
data['state'] = pd.Series(final_list)
data['state'].value_counts(dropna=False)

Remote     565
CA         270
NY         232
IL         113
MA          93
TX          77
WA          66
VA          66
MI          62
PA          58
CO          48
MD          45
NC          43
OH          40
UT          38
NJ          37
SC          29
MO          28
FL          28
GA          27
NM          19
MN          10
DC          10
AZ          10
WI           7
CT           7
OK           6
DE           6
LA           6
IA           5
Unknown      5
ID           5
IN           4
OR           3
TN           3
AR           3
NV           3
NE           1
RI           1
KS           1
AL           1
MS           1
AF           1
PR           1
Name: state, dtype: int64

**There are still some locations defined like 'United States' and e don't know exatly to which state belong, therefore let's make some research:**

In [14]:
data[data['state']=='Unknown']

,company,job_title,location,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,hourly,rating,python_yn,spark_yn,azure_yn,aws_yn,excel_yn,machine_learning_yn,job_simpl,seniority,description_len,company_age,state
1292,MCG Health,Data Engineer,United States,"At MCG, we lead the healthcare community to de...",135800,201 to 500 Employees,Subsidiary or Business Segment,Healthcare,Health Care Services & Hospitals,1988.0,Unknown / Non-Applicable,0,4.6,1,1,1,1,0,0,data engineer,Senior,347,34.0,Unknown
1342,MCG Health,Data Engineer,United States,"At MCG, we lead the healthcare community to de...",135800,201 to 500 Employees,Subsidiary or Business Segment,Healthcare,Health Care Services & Hospitals,1988.0,Unknown / Non-Applicable,0,4.6,1,1,1,1,0,0,data engineer,Senior,347,34.0,Unknown
1459,Boom Entertainment,Data Engineer - Remote,United States,Boom Entertainment is a technology company tha...,117500,51 to 200 Employees,Company - Private,Information Technology,Information Technology Support Services,2015.0,Unknown / Non-Applicable,0,4.4,1,0,0,0,0,0,data engineer,Senior,412,7.0,Unknown
1538,MCG Health,Data Engineer,United States,"At MCG, we lead the healthcare community to de...",135800,201 to 500 Employees,Subsidiary or Business Segment,Healthcare,Health Care Services & Hospitals,1988.0,Unknown / Non-Applicable,0,4.6,1,1,1,1,0,0,data engineer,Senior,347,34.0,Unknown
1564,MCG Health,Data Engineer,United States,"At MCG, we lead the healthcare community to de...",135800,201 to 500 Employees,Subsidiary or Business Segment,Healthcare,Health Care Services & Hospitals,1988.0,Unknown / Non-Applicable,0,4.6,1,1,1,1,0,0,data engineer,Senior,345,34.0,Unknown


Searching on the internet we can check that the company MCG Health is mostly located in Seattle, Washington; while the offices of Boom Entertainment are located in New York, therefore we are going to assume that these unknown locations are goning to be close to the offices of both companies.

In [15]:
data.loc[1459,'state'] = data.loc[1459,'state'].replace('Unknown','NY')

In [16]:
data['state'] = data['state'].replace('Unknown','WA')

## Dealing with NaN values

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2084 entries, 0 to 2083
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              2084 non-null   object 
 1   job_title            2084 non-null   object 
 2   location             2084 non-null   object 
 3   job_description      2084 non-null   object 
 4   salary_estimate      2084 non-null   int64  
 5   company_size         1917 non-null   object 
 6   company_type         1917 non-null   object 
 7   company_sector       1568 non-null   object 
 8   company_industry     1568 non-null   object 
 9   company_founded      1383 non-null   float64
 10  company_revenue      1917 non-null   object 
 11  hourly               2084 non-null   int64  
 12  rating               1711 non-null   float64
 13  python_yn            2084 non-null   int64  
 14  spark_yn             2084 non-null   int64  
 15  azure_yn             2084 non-null   i

**Which are the companies that have NaN values?**

In [18]:
data_nan = data[data['company_size'].isna()]
data_nan['company'].value_counts(dropna=False)

Ingress IT Solution                                                                  27
STEPS TALENT, LLC                                                                    18
AIVantage INC                                                                        17
Mican Technologies                                                                   10
Tekwissenll                                                                           6
Enterprise Mind                                                                       6
v-solv                                                                                6
Seha Consultin                                                                        6
GRI                                                                                   5
Teamflo                                                                               5
Leading Edge Skill                                                                    5
Lumin Globa                     

In [19]:
company_nan = data_nan['company'].value_counts(dropna=False).index.tolist()
len(company_nan)

42

In [20]:
company_cols= ['company_size','company_type', 'company_sector','company_industry', 'company_founded', 'company_revenue', 'company_age']

for col in company_cols:
    print(data[col].value_counts(dropna=False))

10000+ Employees           576
1001 to 5000 Employees     343
1 to 50 Employees          295
Unknown                    256
201 to 500 Employees       186
NaN                        167
51 to 200 Employees        137
501 to 1000 Employees       67
5001 to 10000 Employees     57
Name: company_size, dtype: int64
Company - Private                 868
Company - Public                  735
NaN                               167
Subsidiary or Business Segment    108
Nonprofit Organization             54
Government                         51
College / University               43
Hospital                           28
Private Practice / Firm            10
Unknown                             7
Franchise                           7
Contract                            5
Self-employed                       1
Name: company_type, dtype: int64
Information Technology                         521
NaN                                            516
Financial Services                             186
Manufact

In [21]:
data.loc[data['company']=='Ingress IT Solution', 'company_size'] = data.loc[data['company']=='Ingress IT Solution', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Ingress IT Solution', 'company_type'] = data.loc[data['company']=='Ingress IT Solution', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Ingress IT Solution', 'company_sector'] = data.loc[data['company']=='Ingress IT Solution', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='Ingress IT Solution', 'company_founded'] = data.loc[data['company']=='Ingress IT Solution', 'company_founded'].fillna(2012) 
data.loc[data['company']=='Ingress IT Solution', 'company_revenue'] = data.loc[data['company']=='Ingress IT Solution', 'company_revenue'].fillna('$1 to $5 million (USD)')
data.loc[data['company']=='Ingress IT Solution', 'company_industry'] = data.loc[data['company']=='Ingress IT Solution', 'company_industry'].fillna('Information Technology Support Services') 
data.loc[data['company']=='Ingress IT Solution', 'company_age'] = data.loc[data['company']=='Ingress IT Solution', 'company_age'].fillna(10)
data.loc[data['company']=='Ingress IT Solution', 'rating'] = data.loc[data['company']=='Ingress IT Solution', 'rating'].fillna(4.2)

In [22]:
data.loc[data['company']=='STEPS TALENT, LLC', 'company_size'] = data.loc[data['company']=='STEPS TALENT, LLC', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='STEPS TALENT, LLC', 'company_type'] = data.loc[data['company']=='STEPS TALENT, LLC', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='STEPS TALENT, LLC', 'company_sector'] = data.loc[data['company']=='STEPS TALENT, LLC', 'company_sector'].fillna('Human Resources & Staffing')
data.loc[data['company']=='STEPS TALENT, LLC', 'company_founded'] = data.loc[data['company']=='STEPS TALENT, LLC', 'company_founded'].fillna(2022) 
data.loc[data['company']=='STEPS TALENT, LLC', 'company_revenue'] = data.loc[data['company']=='STEPS TALENT, LLC', 'company_revenue'].fillna('$1 to $5 million (USD)')
data.loc[data['company']=='STEPS TALENT, LLC', 'company_industry'] = data.loc[data['company']=='STEPS TALENT, LLC', 'company_industry'].fillna('HR Consulting') 
data.loc[data['company']=='STEPS TALENT, LLC', 'company_age'] = data.loc[data['company']=='STEPS TALENT, LLC', 'company_age'].fillna(0)

In [23]:
data.loc[data['company']=='AIVantage INC', 'company_size'] = data.loc[data['company']=='AIVantage INC', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='AIVantage INC', 'company_type'] = data.loc[data['company']=='AIVantage INC', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='AIVantage INC', 'company_sector'] = data.loc[data['company']=='AIVantage INC', 'company_sector'].fillna('Management & Consulting')
data.loc[data['company']=='AIVantage INC', 'company_founded'] = data.loc[data['company']=='AIVantage INC', 'company_founded'].fillna(2022) 
data.loc[data['company']=='AIVantage INC', 'company_revenue'] = data.loc[data['company']=='AIVantage INC', 'company_revenue'].fillna('$1 to $5 million (USD)')
data.loc[data['company']=='AIVantage INC', 'company_industry'] = data.loc[data['company']=='AIVantage INC', 'company_industry'].fillna('Business Consulting') 
data.loc[data['company']=='AIVantage INC', 'company_age'] = data.loc[data['company']=='AIVantage INC', 'company_age'].fillna(0)

In [24]:
data.loc[data['company']=='Mican Technologies', 'company_size'] = data.loc[data['company']=='Mican Technologies', 'company_size'].fillna('51 to 200 Employees')
data.loc[data['company']=='Mican Technologies', 'company_size'] = data.loc[data['company']=='Mican Technologies', 'company_size'].replace('Unknown', '51 to 200 Employees')
data.loc[data['company']=='Mican Technologies', 'company_type'] = data.loc[data['company']=='Mican Technologies', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Mican Technologies', 'company_sector'] = data.loc[data['company']=='Mican Technologies', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='Mican Technologies', 'company_founded'] = data.loc[data['company']=='Mican Technologies', 'company_founded'].fillna(2007) 
data.loc[data['company']=='Mican Technologies', 'company_revenue'] = data.loc[data['company']=='Mican Technologies', 'company_revenue'].fillna('$5 to $25 million (USD)')
data.loc[data['company']=='Mican Technologies', 'company_industry'] = data.loc[data['company']=='Mican Technologies', 'company_industry'].fillna('Information Technology Support Services') 
data.loc[data['company']=='Mican Technologies', 'company_age'] = data.loc[data['company']=='Mican Technologies', 'company_age'].fillna(15)
data.loc[data['company']=='Mican Technologies', 'rating'] = data.loc[data['company']=='Mican Technologies', 'rating'].fillna(4.3)

In [25]:
data.loc[data['company']=='Tekwissenll', 'company_size'] = data.loc[data['company']=='Tekwissenll', 'company_size'].fillna('501 to 1000 Employees')
data.loc[data['company']=='Tekwissenll', 'company_type'] = data.loc[data['company']=='Tekwissenll', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Tekwissenll', 'company_sector'] = data.loc[data['company']=='Tekwissenll', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='Tekwissenll', 'company_founded'] = data.loc[data['company']=='Tekwissenll', 'company_founded'].fillna(2009) 
data.loc[data['company']=='Tekwissenll', 'company_revenue'] = data.loc[data['company']=='Tekwissenll', 'company_revenue'].fillna('$25 to $100 million (USD)')
data.loc[data['company']=='Tekwissenll', 'company_industry'] = data.loc[data['company']=='Tekwissenll', 'company_industry'].fillna('Information Technology Support Services') 
data.loc[data['company']=='Tekwissenll', 'company_age'] = data.loc[data['company']=='Tekwissenll', 'company_age'].fillna(13)
data.loc[data['company']=='Tekwissenll', 'rating'] = data.loc[data['company']=='Tekwissenll', 'rating'].fillna(4.3)
data['company'] = data['company'].replace('Tekwissenll', 'Tekwissen') 

In [26]:
data.loc[data['company']=='Enterprise Mind', 'company_size'] = data.loc[data['company']=='Enterprise Mind', 'company_size'].fillna('201 to 500 Employees')
data.loc[data['company']=='Enterprise Mind', 'company_type'] = data.loc[data['company']=='Enterprise Mind', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Enterprise Mind', 'company_sector'] = data.loc[data['company']=='Enterprise Mind', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='Enterprise Mind', 'company_founded'] = data.loc[data['company']=='Enterprise Mind', 'company_founded'].fillna(2017) 
data.loc[data['company']=='Enterprise Mind', 'company_revenue'] = data.loc[data['company']=='Enterprise Mind', 'company_revenue'].fillna('$1 to $5 million (USD)')
data.loc[data['company']=='Enterprise Mind', 'company_industry'] = data.loc[data['company']=='Enterprise Mind', 'company_industry'].fillna('Information Technology Support Services') 
data.loc[data['company']=='Enterprise Mind', 'company_age'] = data.loc[data['company']=='Enterprise Mind', 'company_age'].fillna(5)
data.loc[data['company']=='Enterprise Mind', 'rating'] = data.loc[data['company']=='Enterprise Mind', 'rating'].fillna(4.)
data['company'] = data['company'].replace('Enterprise Mind', 'Enterprise Minds')

In [27]:
data.loc[data['company']=='v-solv', 'company_size'] = data.loc[data['company']=='v-solv', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='v-solv', 'company_type'] = data.loc[data['company']=='v-solv', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='v-solv', 'company_sector'] = data.loc[data['company']=='v-solv', 'company_sector'].fillna('Financial Services')
data.loc[data['company']=='v-solv', 'company_founded'] = data.loc[data['company']=='v-solv', 'company_founded'].fillna(2009) 
data.loc[data['company']=='v-solv', 'company_revenue'] = data.loc[data['company']=='v-solv', 'company_revenue'].fillna('Less than $1 million (USD)')
data.loc[data['company']=='v-solv', 'company_industry'] = data.loc[data['company']=='v-solv', 'company_industry'].fillna('Information Technology Support Services') 
data.loc[data['company']=='v-solv', 'company_age'] = data.loc[data['company']=='v-solv', 'company_age'].fillna(13)
data.loc[data['company']=='v-solv', 'rating'] = data.loc[data['company']=='v-solv', 'rating'].fillna(4.)
data['company'] = data['company'].replace('v-solv', 'v-solve') 

In [28]:
data.loc[data['company']=='Seha Consultin', 'company_size'] = data.loc[data['company']=='Seha Consultin', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Seha Consultin', 'company_type'] = data.loc[data['company']=='Seha Consultin', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Seha Consultin', 'company_sector'] = data.loc[data['company']=='Seha Consultin', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='Seha Consultin', 'company_founded'] = data.loc[data['company']=='Seha Consultin', 'company_founded'].fillna(2021) 
data.loc[data['company']=='Seha Consultin', 'company_revenue'] = data.loc[data['company']=='Seha Consultin', 'company_revenue'].fillna('Less than $1 million (USD)')
data.loc[data['company']=='Seha Consultin', 'company_industry'] = data.loc[data['company']=='Seha Consultin', 'company_industry'].fillna('Business consulting') 
data.loc[data['company']=='Seha Consultin', 'company_age'] = data.loc[data['company']=='Seha Consultin', 'company_age'].fillna(1)
data['company'] = data['company'].replace('Seha Consultin', 'Seha Consulting')

In [29]:
data.loc[data['company']=='GRI', 'company_size'] = data.loc[data['company']=='GRI', 'company_size'].fillna('51 to 200 Employees')
data.loc[data['company']=='GRI', 'company_type'] = data.loc[data['company']=='GRI', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='GRI', 'company_sector'] = data.loc[data['company']=='GRI', 'company_sector'].fillna('Construction, Repair & Maintenance Services')
data.loc[data['company']=='GRI', 'company_founded'] = data.loc[data['company']=='GRI', 'company_founded'].fillna(2014) 
data.loc[data['company']=='GRI', 'company_revenue'] = data.loc[data['company']=='GRI', 'company_revenue'].fillna('$100 to $500 million (USD)')
data.loc[data['company']=='GRI', 'company_industry'] = data.loc[data['company']=='GRI', 'company_industry'].fillna('Architectural & Engineering Services') 
data.loc[data['company']=='GRI', 'company_age'] = data.loc[data['company']=='GRI', 'company_age'].fillna(8)
data.loc[data['company']=='GRI', 'rating'] = data.loc[data['company']=='GRI', 'rating'].fillna(4.1) 

In [30]:
data.loc[data['company']=='Teamflo', 'company_size'] = data.loc[data['company']=='Teamflo', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Teamflo', 'company_type'] = data.loc[data['company']=='Teamflo', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Teamflo', 'company_sector'] = data.loc[data['company']=='Teamflo', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='Teamflo', 'company_founded'] = data.loc[data['company']=='Teamflo', 'company_founded'].fillna(2020) 
data.loc[data['company']=='Teamflo', 'company_revenue'] = data.loc[data['company']=='Teamflo', 'company_revenue'].fillna('$1 to $5 billion (USD)')
data.loc[data['company']=='Teamflo', 'company_industry'] = data.loc[data['company']=='Teamflo', 'company_industry'].fillna('Internet & Web Services') 
data.loc[data['company']=='Teamflo', 'company_age'] = data.loc[data['company']=='Teamflo', 'company_age'].fillna(2)
data['company'] = data['company'].replace('Teamflo', 'Teamflow') 

In [31]:
data.loc[data['company']=='Leading Edge Skill', 'company_size'] = data.loc[data['company']=='Leading Edge Skill', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Leading Edge Skill', 'company_type'] = data.loc[data['company']=='Leading Edge Skill', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Leading Edge Skill', 'company_sector'] = data.loc[data['company']=='Leading Edge Skill', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='Leading Edge Skill', 'company_founded'] = data.loc[data['company']=='Leading Edge Skill', 'company_founded'].fillna(2020) 
data.loc[data['company']=='Leading Edge Skill', 'company_revenue'] = data.loc[data['company']=='Leading Edge Skill', 'company_revenue'].fillna('$1 to $5 billion (USD)')
data.loc[data['company']=='Leading Edge Skill', 'company_industry'] = data.loc[data['company']=='Leading Edge Skill', 'company_industry'].fillna('Education & Training Services') 
data.loc[data['company']=='Leading Edge Skill', 'company_age'] = data.loc[data['company']=='Leading Edge Skill', 'company_age'].fillna(2)
data['company'] = data['company'].replace('Leading Edge Skill', 'Leading Edge Skills')

In [32]:
data.loc[data['company']=='Lumin Globa', 'company_size'] = data.loc[data['company']=='Lumin Globa', 'company_size'].fillna('51 to 200 Employees')
data.loc[data['company']=='Lumin Globa', 'company_type'] = data.loc[data['company']=='Lumin Globa', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Lumin Globa', 'company_sector'] = data.loc[data['company']=='Lumin Globa', 'company_sector'].fillna('Energy, Mining & Utilities')
data.loc[data['company']=='Lumin Globa', 'company_founded'] = data.loc[data['company']=='Lumin Globa', 'company_founded'].fillna(2016) 
data.loc[data['company']=='Lumin Globa', 'company_revenue'] = data.loc[data['company']=='Lumin Globa', 'company_revenue'].fillna('$1 to $5 billion (USD)')
data.loc[data['company']=='Lumin Globa', 'company_industry'] = data.loc[data['company']=='Lumin Globa', 'company_industry'].fillna('Energy & Utilities') 
data.loc[data['company']=='Lumin Globa', 'company_age'] = data.loc[data['company']=='Lumin Globa', 'company_age'].fillna(6)
data.loc[data['company']=='Lumin Globa', 'rating'] = data.loc[data['company']=='Lumin Globa', 'rating'].fillna(4.8)
data['company'] = data['company'].replace('Lumin Globa', 'Lumin Global')

In [33]:
data.loc[data['company']=='The New York City Post LL', 'company_size'] = data.loc[data['company']=='The New York City Post LL', 'company_size'].fillna('1001 to 5000 Employees')
data.loc[data['company']=='The New York City Post LL', 'company_type'] = data.loc[data['company']=='The New York City Post LL', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='The New York City Post LL', 'company_sector'] = data.loc[data['company']=='The New York City Post LL', 'company_sector'].fillna('Media & Communication')
data.loc[data['company']=='The New York City Post LL', 'company_founded'] = data.loc[data['company']=='The New York City Post LL', 'company_founded'].fillna(1801) 
data.loc[data['company']=='The New York City Post LL', 'company_revenue'] = data.loc[data['company']=='The New York City Post LL', 'company_revenue'].fillna('$100 to $500 million (USD)')
data.loc[data['company']=='The New York City Post LL', 'company_industry'] = data.loc[data['company']=='The New York City Post LL', 'company_industry'].fillna('Publishing') 
data.loc[data['company']=='The New York City Post LL', 'company_age'] = data.loc[data['company']=='The New York City Post LL', 'company_age'].fillna(221)
data.loc[data['company']=='The New York City Post LL', 'rating'] = data.loc[data['company']=='The New York City Post LL', 'rating'].fillna(4.)
data['company'] = data['company'].replace('The New York City Post LL', 'The New York City Post LLC')

In [34]:
data.loc[data['company']=='Choice ', 'company_size'] = data.loc[data['company']=='Choice ', 'company_size'].fillna('1001 to 5000 Employees')
data.loc[data['company']=='Choice ', 'company_type'] = data.loc[data['company']=='Choice ', 'company_type'].fillna('Company - Public')
data.loc[data['company']=='Choice ', 'company_sector'] = data.loc[data['company']=='Choice ', 'company_sector'].fillna('Hotels & Travel Accommodation')
data.loc[data['company']=='Choice ', 'company_founded'] = data.loc[data['company']=='Choice ', 'company_founded'].fillna(1939) 
data.loc[data['company']=='Choice ', 'company_revenue'] = data.loc[data['company']=='Choice ', 'company_revenue'].fillna('$1 to $5 billion (USD)')
data.loc[data['company']=='Choice ', 'company_industry'] = data.loc[data['company']=='Choice ', 'company_industry'].fillna('Hotels & Resorts') 
data.loc[data['company']=='Choice ', 'company_age'] = data.loc[data['company']=='Choice ', 'company_age'].fillna(83)
data.loc[data['company']=='Choice ', 'rating'] = data.loc[data['company']=='Choice ', 'rating'].fillna(4.)
data['company'] = data['company'].replace('Choice ', 'Choice Hotels')

In [35]:
data.loc[data['company']=='BlueCondui', 'company_size'] = data.loc[data['company']=='BlueCondui', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='BlueCondui', 'company_type'] = data.loc[data['company']=='BlueCondui', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='BlueCondui', 'company_sector'] = data.loc[data['company']=='BlueCondui', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='BlueCondui', 'company_founded'] = data.loc[data['company']=='BlueCondui', 'company_founded'].fillna(2019) 
data.loc[data['company']=='BlueCondui', 'company_revenue'] = data.loc[data['company']=='BlueCondui', 'company_revenue'].fillna('$1 to $5 billion (USD)')
data.loc[data['company']=='BlueCondui', 'company_industry'] = data.loc[data['company']=='BlueCondui', 'company_industry'].fillna('Computer Hardware Development') 
data.loc[data['company']=='BlueCondui', 'company_age'] = data.loc[data['company']=='BlueCondui', 'company_age'].fillna(3)
data.loc[data['company']=='BlueCondui', 'rating'] = data.loc[data['company']=='BlueCondui', 'rating'].fillna(4.6)
data['company'] = data['company'].replace('BlueCondui', 'BlueConduit')

In [36]:
data.loc[data['company']=='Generation Genius, Inc', 'company_size'] = data.loc[data['company']=='Generation Genius, Inc', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Generation Genius, Inc', 'company_type'] = data.loc[data['company']=='Generation Genius, Inc', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Generation Genius, Inc', 'company_sector'] = data.loc[data['company']=='Generation Genius, Inc', 'company_sector'].fillna('Education')
data.loc[data['company']=='Generation Genius, Inc', 'company_founded'] = data.loc[data['company']=='Generation Genius, Inc', 'company_founded'].fillna(2017) 
data.loc[data['company']=='Generation Genius, Inc', 'company_revenue'] = data.loc[data['company']=='Generation Genius, Inc', 'company_revenue'].fillna('$1 to $5 billion (USD)')
data.loc[data['company']=='Generation Genius, Inc', 'company_industry'] = data.loc[data['company']=='Generation Genius, Inc', 'company_industry'].fillna('Education & Training Services') 
data.loc[data['company']=='Generation Genius, Inc', 'company_age'] = data.loc[data['company']=='Generation Genius, Inc', 'company_age'].fillna(5)

In [37]:
data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_size'] = data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_type'] = data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_sector'] = data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_sector'].fillna('Management & Consulting')
data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_founded'] = data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_founded'].fillna(2017) 
data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_revenue'] = data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_revenue'].fillna('Less than $1 million (USD)')
data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_industry'] = data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_industry'].fillna('Business Consulting') 
data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_age'] = data.loc[data['company']=='NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'company_age'].fillna(5)
data['salary_estimate'] = data['salary_estimate'].replace(3760, 3760*12)
data['company'] = data['company'].replace('NEXT LEVEL BUSINESS CONCEPTS GROUP LL', 'NEXT LEVEL BUSINESS CONCEPTS GROUP LLC') 

In [38]:
data.loc[data['company']=='HealthAware LL', 'company_size'] = data.loc[data['company']=='HealthAware LL', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='HealthAware LL', 'company_type'] = data.loc[data['company']=='HealthAware LL', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='HealthAware LL', 'company_sector'] = data.loc[data['company']=='HealthAware LL', 'company_sector'].fillna('Healthcare')
data.loc[data['company']=='HealthAware LL', 'company_founded'] = data.loc[data['company']=='HealthAware LL', 'company_founded'].fillna(2004) 
data.loc[data['company']=='HealthAware LL', 'company_revenue'] = data.loc[data['company']=='HealthAware LL', 'company_revenue'].fillna('$5 to $25 million (USD)')
data.loc[data['company']=='HealthAware LL', 'company_industry'] = data.loc[data['company']=='HealthAware LL', 'company_industry'].fillna('Health Care Services & Hospitals') 
data.loc[data['company']=='HealthAware LL', 'company_age'] = data.loc[data['company']=='HealthAware LL', 'company_age'].fillna(18)
data.loc[data['company']=='HealthAware LL', 'rating'] = data.loc[data['company']=='HealthAware LL', 'rating'].fillna(4.1)
data['company'] = data['company'].replace('HealthAware LL', 'HealthAware LLC')

In [39]:
data.loc[data['company']=='Ritwik Infotec', 'company_size'] = data.loc[data['company']=='Ritwik Infotec', 'company_size'].fillna('51 to 200 Employees')
data.loc[data['company']=='Ritwik Infotec', 'company_type'] = data.loc[data['company']=='Ritwik Infotec', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Ritwik Infotec', 'company_sector'] = data.loc[data['company']=='Ritwik Infotec', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='Ritwik Infotec', 'company_founded'] = data.loc[data['company']=='Ritwik Infotec', 'company_founded'].fillna(2015) 
data.loc[data['company']=='Ritwik Infotec', 'company_revenue'] = data.loc[data['company']=='Ritwik Infotec', 'company_revenue'].fillna('$5 to $25 million (USD)')
data.loc[data['company']=='Ritwik Infotec', 'company_industry'] = data.loc[data['company']=='Ritwik Infotec', 'company_industry'].fillna('Information Technology Support Services') 
data.loc[data['company']=='Ritwik Infotec', 'company_age'] = data.loc[data['company']=='Ritwik Infotec', 'company_age'].fillna(7)
data.loc[data['company']=='Ritwik Infotec', 'rating'] = data.loc[data['company']=='Ritwik Infotec', 'rating'].fillna(2.7)
data['company'] = data['company'].replace('Ritwik Infotec', 'Ritwik Infotech')

In [40]:
data['company'] = data['company'].replace('archontulsaitrecruiters@gmail.co', 'Archon Resources')
data.loc[data['company']=='Archon Resources', 'company_size'] = data.loc[data['company']=='Archon Resources', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Archon Resources', 'company_type'] = data.loc[data['company']=='Archon Resources', 'company_type'].fillna('Company - Public')
data.loc[data['company']=='Archon Resources', 'company_sector'] = data.loc[data['company']=='Archon Resources', 'company_sector'].fillna('Human Resources & Staffing')
data.loc[data['company']=='Archon Resources', 'company_founded'] = data.loc[data['company']=='Archon Resources', 'company_founded'].fillna(2018) 
data.loc[data['company']=='Archon Resources', 'company_revenue'] = data.loc[data['company']=='Archon Resources', 'company_revenue'].fillna('$25 to $100 million (USD)')
data.loc[data['company']=='Archon Resources', 'company_industry'] = data.loc[data['company']=='Archon Resources', 'company_industry'].fillna('Staffing & Subcontracting') 
data.loc[data['company']=='Archon Resources', 'company_age'] = data.loc[data['company']=='Archon Resources', 'company_age'].fillna(4)
data.loc[data['company']=='Archon Resources', 'rating'] = data.loc[data['company']=='Archon Resources', 'rating'].fillna(3.6)

In [41]:
data.loc[data['company']=='Summit Materia', 'company_size'] = data.loc[data['company']=='Summit Materia', 'company_size'].fillna('201 to 500 Employees')
data.loc[data['company']=='Summit Materia', 'company_type'] = data.loc[data['company']=='Summit Materia', 'company_type'].fillna('Company - Public')
data.loc[data['company']=='Summit Materia', 'company_sector'] = data.loc[data['company']=='Summit Materia', 'company_sector'].fillna('Energy, Mining & Utilities')
data.loc[data['company']=='Summit Materia', 'company_founded'] = data.loc[data['company']=='Summit Materia', 'company_founded'].fillna(2009) 
data.loc[data['company']=='Summit Materia', 'company_revenue'] = data.loc[data['company']=='Summit Materia', 'company_revenue'].fillna('$25 to $100 million (USD)')
data.loc[data['company']=='Summit Materia', 'company_industry'] = data.loc[data['company']=='Summit Materia', 'company_industry'].fillna('Wholesale') 
data.loc[data['company']=='Summit Materia', 'company_age'] = data.loc[data['company']=='Summit Materia', 'company_age'].fillna(13)
data.loc[data['company']=='Summit Materia', 'rating'] = data.loc[data['company']=='Summit Materia', 'rating'].fillna(3.6)
data['company'] = data['company'].replace('Summit Materia', 'Summit Materials')

In [42]:
data.loc[data['company']=='Core Alliance Group Inc', 'company_size'] = data.loc[data['company']=='Core Alliance Group Inc', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Core Alliance Group Inc', 'company_type'] = data.loc[data['company']=='Core Alliance Group Inc', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Core Alliance Group Inc', 'company_sector'] = data.loc[data['company']=='Core Alliance Group Inc', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='Core Alliance Group Inc', 'company_founded'] = data.loc[data['company']=='Core Alliance Group Inc', 'company_founded'].fillna(2005) 
data.loc[data['company']=='Core Alliance Group Inc', 'company_revenue'] = data.loc[data['company']=='Core Alliance Group Inc', 'company_revenue'].fillna('$1 to $5 million (USD)')
data.loc[data['company']=='Core Alliance Group Inc', 'company_industry'] = data.loc[data['company']=='Core Alliance Group Inc', 'company_industry'].fillna('Information Technology Support Services') 
data.loc[data['company']=='Core Alliance Group Inc', 'company_age'] = data.loc[data['company']=='Core Alliance Group Inc', 'company_age'].fillna(17)

In [43]:
data.loc[data['company']=='Avenir Healt', 'company_size'] = data.loc[data['company']=='Avenir Healt', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Avenir Healt', 'company_type'] = data.loc[data['company']=='Avenir Healt', 'company_type'].fillna('Nonprofit Organization')
data.loc[data['company']=='Avenir Healt', 'company_sector'] = data.loc[data['company']=='Avenir Healt', 'company_sector'].fillna('Healthcare')
data.loc[data['company']=='Avenir Healt', 'company_founded'] = data.loc[data['company']=='Avenir Healt', 'company_founded'].fillna(2006) 
data.loc[data['company']=='Avenir Healt', 'company_revenue'] = data.loc[data['company']=='Avenir Healt', 'company_revenue'].fillna('$5 to $25 million (USD)')
data.loc[data['company']=='Avenir Healt', 'company_industry'] = data.loc[data['company']=='Avenir Healt', 'company_industry'].fillna('Health Care Services & Hospitals') 
data.loc[data['company']=='Avenir Healt', 'company_age'] = data.loc[data['company']=='Avenir Healt', 'company_age'].fillna(16)
data['company'] = data['company'].replace('Avenir Healt', 'Avenir Health')

In [44]:
data.loc[data['company']=='TEKtalent In', 'company_size'] = data.loc[data['company']=='TEKtalent In', 'company_size'].fillna('51 to 200 Employees')
data.loc[data['company']=='TEKtalent In', 'company_type'] = data.loc[data['company']=='TEKtalent In', 'company_type'].fillna('Company - Public')
data.loc[data['company']=='TEKtalent In', 'company_sector'] = data.loc[data['company']=='TEKtalent In', 'company_sector'].fillna('Human Resources & Staffing')
data.loc[data['company']=='TEKtalent In', 'company_founded'] = data.loc[data['company']=='TEKtalent In', 'company_founded'].fillna(2000) 
data.loc[data['company']=='TEKtalent In', 'company_revenue'] = data.loc[data['company']=='TEKtalent In', 'company_revenue'].fillna('$5 to $25 million (USD)')
data.loc[data['company']=='TEKtalent In', 'company_industry'] = data.loc[data['company']=='TEKtalent In', 'company_industry'].fillna('Staffing & Subcontracting') 
data.loc[data['company']=='TEKtalent In', 'rating'] = data.loc[data['company']=='TEKtalent In', 'rating'].fillna(3.)
data.loc[data['company']=='TEKtalent In', 'company_age'] = data.loc[data['company']=='TEKtalent In', 'company_age'].fillna(22)
data['company'] = data['company'].replace('TEKtalent In', 'TEKtalent Inc')

In [45]:
data['company'] = data['company'].replace('Koo Lab at Cold Spring Harbor Laborator', 'Cold Spring Harbor Laboratory')
data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_size'] = data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_size'].fillna('51 to 200 Employees')
data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_type'] = data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_type'].fillna('Nonprofit Organization')
data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_sector'] = data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_sector'].fillna('Nonprofit & NGO')
data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_founded'] = data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_founded'].fillna(1890) 
data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_revenue'] = data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_revenue'].fillna('$5 to $25 million (USD)')
data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_industry'] = data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_industry'].fillna('Research & Development') 
data.loc[data['company']=='Cold Spring Harbor Laboratory', 'rating'] = data.loc[data['company']=='Cold Spring Harbor Laboratory', 'rating'].fillna(3.9)
data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_age'] = data.loc[data['company']=='Cold Spring Harbor Laboratory', 'company_age'].fillna(132)

In [46]:
data.loc[data['company']=='Sceye', 'company_size'] = data.loc[data['company']=='Sceye', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Sceye', 'company_type'] = data.loc[data['company']=='Sceye', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Sceye', 'company_sector'] = data.loc[data['company']=='Sceye', 'company_sector'].fillna('Aerospace & Defense')
data.loc[data['company']=='Sceye', 'company_founded'] = data.loc[data['company']=='Sceye', 'company_founded'].fillna(2014) 
data.loc[data['company']=='Sceye', 'company_revenue'] = data.loc[data['company']=='Sceye', 'company_revenue'].fillna('$1 to $5 million (USD)')
data.loc[data['company']=='Sceye', 'company_industry'] = data.loc[data['company']=='Sceye', 'company_industry'].fillna('Transportation Equipment Manufacturing') 
data.loc[data['company']=='Sceye', 'company_age'] = data.loc[data['company']=='Sceye', 'company_age'].fillna(8)

In [47]:
data.loc[data['company']=='Attunely', 'company_size'] = data.loc[data['company']=='Attunely', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Attunely', 'company_type'] = data.loc[data['company']=='Attunely', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Attunely', 'company_sector'] = data.loc[data['company']=='Attunely', 'company_sector'].fillna('Financial Services')
data.loc[data['company']=='Attunely', 'company_founded'] = data.loc[data['company']=='Attunely', 'company_founded'].fillna(2018) 
data.loc[data['company']=='Attunely', 'company_revenue'] = data.loc[data['company']=='Attunely', 'company_revenue'].fillna('$1 to $5 million (USD)')
data.loc[data['company']=='Attunely', 'company_industry'] = data.loc[data['company']=='Attunely', 'company_industry'].fillna('Computer Hardware Development') 
data.loc[data['company']=='Attunely', 'rating'] = data.loc[data['company']=='Attunely', 'rating'].fillna(5.)
data.loc[data['company']=='Attunely', 'company_age'] = data.loc[data['company']=='Attunely', 'company_age'].fillna(4)

In [48]:
data.loc[data['company']=='ACM Analytics, LL', 'company_size'] = data.loc[data['company']=='ACM Analytics, LL', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='ACM Analytics, LL', 'company_type'] = data.loc[data['company']=='ACM Analytics, LL', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='ACM Analytics, LL', 'company_sector'] = data.loc[data['company']=='ACM Analytics, LL', 'company_sector'].fillna('Management & Consulting')
data.loc[data['company']=='ACM Analytics, LL', 'company_founded'] = data.loc[data['company']=='ACM Analytics, LL', 'company_founded'].fillna(2018) 
data.loc[data['company']=='ACM Analytics, LL', 'company_revenue'] = data.loc[data['company']=='ACM Analytics, LL', 'company_revenue'].fillna('Less than $1 million (USD)')
data.loc[data['company']=='ACM Analytics, LL', 'company_industry'] = data.loc[data['company']=='ACM Analytics, LL', 'company_industry'].fillna('Business Consulting') 
data.loc[data['company']=='ACM Analytics, LL', 'company_age'] = data.loc[data['company']=='ACM Analytics, LL', 'company_age'].fillna(4)
data['company'] = data['company'].replace('ACM Analytics, LL', 'ACM Analytics, LLC')

In [49]:
data.loc[data['company']=='Talentheed In', 'company_size'] = data.loc[data['company']=='Talentheed In', 'company_size'].fillna('51 to 200 Employees')
data.loc[data['company']=='Talentheed In', 'company_type'] = data.loc[data['company']=='Talentheed In', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Talentheed In', 'company_sector'] = data.loc[data['company']=='Talentheed In', 'company_sector'].fillna('Management & Consulting')
data.loc[data['company']=='Talentheed In', 'company_founded'] = data.loc[data['company']=='Talentheed In', 'company_founded'].fillna(2016) 
data.loc[data['company']=='Talentheed In', 'company_revenue'] = data.loc[data['company']=='Talentheed In', 'company_revenue'].fillna('$1 to $5 million (USD)')
data.loc[data['company']=='Talentheed In', 'company_industry'] = data.loc[data['company']=='Talentheed In', 'company_industry'].fillna('Business Consulting') 
data.loc[data['company']=='Talentheed In', 'company_age'] = data.loc[data['company']=='Talentheed In', 'company_age'].fillna(6)
data['company'] = data['company'].replace('Talentheed In', 'Talentheed Inc')

In [50]:
data['company'] = data['company'].replace('bitsIO Inc., 920 South Spring St, Ste 1200, Springfield, Illinois 6270', 'bitsIO Inc')
data.loc[data['company']=='bitsIO Inc', 'company_size'] = data.loc[data['company']=='bitsIO Inc', 'company_size'].fillna('51 to 200 Employees')
data.loc[data['company']=='bitsIO Inc', 'company_type'] = data.loc[data['company']=='bitsIO Inc', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='bitsIO Inc', 'company_sector'] = data.loc[data['company']=='bitsIO Inc', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='bitsIO Inc', 'company_founded'] = data.loc[data['company']=='bitsIO Inc', 'company_founded'].fillna(2001) 
data.loc[data['company']=='bitsIO Inc', 'company_revenue'] = data.loc[data['company']=='bitsIO Inc', 'company_revenue'].fillna('$5 to $25 million (USD)')
data.loc[data['company']=='bitsIO Inc', 'company_industry'] = data.loc[data['company']=='bitsIO Inc', 'company_industry'].fillna('Enterprise Software & Network Solutions') 
data.loc[data['company']=='bitsIO Inc', 'company_age'] = data.loc[data['company']=='bitsIO Inc', 'company_age'].fillna(21)
data.loc[data['company']=='bitsIO Inc', 'rating'] = data.loc[data['company']=='bitsIO Inc', 'rating'].fillna(4.5)

In [51]:
data['company'] = data['company'].replace('Amiti consulting corp', 'Amiti consulting')
data.loc[data['company']=='Amiti consulting', 'company_size'] = data.loc[data['company']=='Amiti consulting', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Amiti consulting', 'company_type'] = data.loc[data['company']=='Amiti consulting', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Amiti consulting', 'company_sector'] = data.loc[data['company']=='Amiti consulting', 'company_sector'].fillna('Human Resources & Staffing')
data.loc[data['company']=='Amiti consulting', 'company_founded'] = data.loc[data['company']=='Amiti consulting', 'company_founded'].fillna(2005) 
data.loc[data['company']=='Amiti consulting', 'company_revenue'] = data.loc[data['company']=='Amiti consulting', 'company_revenue'].fillna('Less than $1 million (USD)')
data.loc[data['company']=='Amiti consulting', 'company_industry'] = data.loc[data['company']=='Amiti consulting', 'company_industry'].fillna('HR Consulting') 
data.loc[data['company']=='Amiti consulting', 'company_age'] = data.loc[data['company']=='Amiti consulting', 'company_age'].fillna(17)
data.loc[data['company']=='Amiti consulting', 'rating'] = data.loc[data['company']=='Amiti consulting', 'rating'].fillna(3.9)

In [52]:
data['company'] = data['company'].replace('Artificial Intelligence Labs at the American Alliance for International Education', 'American Alliance for International Education')
data.loc[data['company']=='American Alliance for International Education', 'company_size'] = data.loc[data['company']=='American Alliance for International Education', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='American Alliance for International Education', 'company_type'] = data.loc[data['company']=='American Alliance for International Education', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='American Alliance for International Education', 'company_sector'] = data.loc[data['company']=='American Alliance for International Education', 'company_sector'].fillna('Education')
data.loc[data['company']=='American Alliance for International Education', 'company_founded'] = data.loc[data['company']=='American Alliance for International Education', 'company_founded'].fillna(1993) 
data.loc[data['company']=='American Alliance for International Education', 'company_revenue'] = data.loc[data['company']=='American Alliance for International Education', 'company_revenue'].fillna('Less than $1 million (USD)')
data.loc[data['company']=='American Alliance for International Education', 'company_industry'] = data.loc[data['company']=='American Alliance for International Education', 'company_industry'].fillna('Colleges & Universities') 
data.loc[data['company']=='American Alliance for International Education', 'company_age'] = data.loc[data['company']=='American Alliance for International Education', 'company_age'].fillna(29)
data.loc[data['company']=='American Alliance for International Education', 'rating'] = data.loc[data['company']=='American Alliance for International Education', 'rating'].fillna(3.)

In [53]:
data['company'] = data['company'].replace('CAT Software Services INC', 'CAT Software Services')
data.loc[data['company']=='CAT Software Services', 'company_size'] = data.loc[data['company']=='CAT Software Services', 'company_size'].fillna('501 to 1000 Employees')
data.loc[data['company']=='CAT Software Services', 'company_type'] = data.loc[data['company']=='CAT Software Services', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='CAT Software Services', 'company_sector'] = data.loc[data['company']=='CAT Software Services', 'company_sector'].fillna('Information Technology')
data.loc[data['company']=='CAT Software Services', 'company_founded'] = data.loc[data['company']=='CAT Software Services', 'company_founded'].fillna(2007) 
data.loc[data['company']=='CAT Software Services', 'company_revenue'] = data.loc[data['company']=='CAT Software Services', 'company_revenue'].fillna('Unknown / Non-Applicable')
data.loc[data['company']=='CAT Software Services', 'company_industry'] = data.loc[data['company']=='CAT Software Services', 'company_industry'].fillna('Enterprise Software & Network Solutions') 
data.loc[data['company']=='CAT Software Services', 'company_age'] = data.loc[data['company']=='CAT Software Services', 'company_age'].fillna(15)
data.loc[data['company']=='CAT Software Services', 'rating'] = data.loc[data['company']=='CAT Software Services', 'rating'].fillna(4.)

In [54]:
data['company'] = data['company'].replace('Two Bears Environmental Consulting LL', 'Two Bears Environmental Consulting')
data.loc[data['company']=='Two Bears Environmental Consulting', 'company_size'] = data.loc[data['company']=='Two Bears Environmental Consulting', 'company_size'].fillna('1 to 50 Employees')
data.loc[data['company']=='Two Bears Environmental Consulting', 'company_type'] = data.loc[data['company']=='Two Bears Environmental Consulting', 'company_type'].fillna('Company - Private')
data.loc[data['company']=='Two Bears Environmental Consulting', 'company_sector'] = data.loc[data['company']=='Two Bears Environmental Consulting', 'company_sector'].fillna('Management & Consulting')
data.loc[data['company']=='Two Bears Environmental Consulting', 'company_founded'] = data.loc[data['company']=='Two Bears Environmental Consulting', 'company_founded'].fillna(2015) 
data.loc[data['company']=='Two Bears Environmental Consulting', 'company_revenue'] = data.loc[data['company']=='Two Bears Environmental Consulting', 'company_revenue'].fillna('Less than $1 million (USD)')
data.loc[data['company']=='Two Bears Environmental Consulting', 'company_industry'] = data.loc[data['company']=='Two Bears Environmental Consulting', 'company_industry'].fillna('Civic & Social Services') 
data.loc[data['company']=='Two Bears Environmental Consulting', 'company_age'] = data.loc[data['company']=='Two Bears Environmental Consulting', 'company_age'].fillna(7)

In [55]:
data.describe()

,salary_estimate,company_founded,hourly,rating,python_yn,spark_yn,azure_yn,aws_yn,excel_yn,machine_learning_yn,description_len,company_age
count,2084.000000,1542.000000,2084.000000,1805.000000,2084.000000,2084.000000,2084.000000,2084.000000,2084.000000,2084.000000,2084.000000,1542.000000
mean,108808.335893,1968.911803,0.166027,4.065042,0.712572,0.117083,0.137716,0.293186,0.367083,0.608445,409.041747,53.088197
std,34337.221547,62.520154,0.372194,0.481282,0.452671,0.321596,0.344684,0.455332,0.482125,0.488215,222.222939,62.520154
min,5162.000000,1625.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.000000,0.000000
25%,88384.500000,1946.000000,0.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,238.000000,10.000000
50%,107747.000000,1993.000000,0.000000,4.100000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,403.000000,29.000000
75%,126495.000000,2012.000000,0.000000,4.300000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,583.000000,76.000000
max,297000.000000,2022.000000,1.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2781.000000,397.000000


**Let's fill as unknown the rest of the companies that has not been found**

In [56]:
data.loc[data['company_type'].isna(), 'company_type'] = data.loc[data['company_type'].isna(), 'company_type'].fillna('Unknown')
data.loc[data['company_sector'].isna(), 'company_sector'] = data.loc[data['company_sector'].isna(), 'company_sector'].fillna('Unknown')
data.loc[data['company_industry'].isna(), 'company_industry'] = data.loc[data['company_industry'].isna(), 'company_industry'].fillna('Unknown')
data.loc[data['company_revenue'].isna(), 'company_revenue'] = data.loc[data['company_revenue'].isna(), 'company_revenue'].fillna('Unknown / Non-Applicable')
data.loc[data['company_size'].isna(), 'company_size'] = data.loc[data['company_size'].isna(), 'company_size'].fillna('Unknown')

In [57]:
data.isna().sum()

company                  0
job_title                0
location                 0
job_description          0
salary_estimate          0
company_size             0
company_type             0
company_sector           0
company_industry         0
company_founded        542
company_revenue          0
hourly                   0
rating                 279
python_yn                0
spark_yn                 0
azure_yn                 0
aws_yn                   0
excel_yn                 0
machine_learning_yn      0
job_simpl                0
seniority                0
description_len          0
company_age            542
state                    0
dtype: int64

In [58]:
data['job_simpl'].value_counts()

data scientist               1197
data analyst                  328
data engineer                 309
machine learning engineer     174
other                          76
Name: job_simpl, dtype: int64

## What is going on with the job positions considered as other?

In [59]:
data[data['job_simpl']=='other']

,company,job_title,location,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,hourly,rating,python_yn,spark_yn,azure_yn,aws_yn,excel_yn,machine_learning_yn,job_simpl,seniority,description_len,company_age,state
545,I28 Technologies,Jr Data Science-333318,"Marysville, MI",Knowledge Data Scientist who possesses a passi...,62500,1 to 50 Employees,Company - Private,Financial Services,Accounting & Tax,NaN,Less than $1 million (USD),0,4.2,0,0,0,0,0,1,other,junior,88,NaN,MI
546,I28 Technologies,Jr Data Science-333318,"Marysville, MI",Knowledge Data Scientist who possesses a passi...,62500,1 to 50 Employees,Company - Private,Financial Services,Accounting & Tax,NaN,Less than $1 million (USD),0,4.2,0,0,0,0,0,1,other,junior,88,NaN,MI
571,Procter & Gamble,Research & Development Engineer Co-op (Associa...,"Boston, MA",Your first step towards an amazing career in R...,93133,10000+ Employees,Company - Public,Manufacturing,Consumer Product Manufacturing,1837.0,$10+ billion (USD),0,4.2,0,0,0,0,0,0,other,junior,665,185.0,MA
593,University of North Carolina at Chapel Hill,Research Scientist,North Carolina,Posting Information\nDepartment\nUNC Inst for ...,61000,1 to 50 Employees,College / University,Education,Colleges & Universities,NaN,$1 to $5 million (USD),0,4.1,1,0,0,0,1,1,other,Senior,466,NaN,NC
621,Procter & Gamble,Research and Development Engineer - Co-op - En...,"Mason, OH",Job Description\nAre you interested in innovat...,100481,10000+ Employees,Company - Public,Manufacturing,Consumer Product Manufacturing,1837.0,$10+ billion (USD),0,4.2,0,0,0,0,0,0,other,Senior,495,185.0,OH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,Hunter Engineering,Data Science Co-Op,"Bridgeton, MO",Overview:\nDo you have a passion for data scie...,88383,1001 to 5000 Employees,Company - Private,Manufacturing,Machinery Manufacturing,1946.0,$500 million to $1 billion (USD),0,4.4,1,0,0,0,1,1,other,Senior,305,76.0,MO
2052,precision technologies corp,Jr UI/UX Designer Training and Placement,Remote,If you want to start your IT career as a UI/UX...,70600,201 to 500 Employees,Company - Private,Information Technology,Information Technology Support Services,2008.0,$5 to $25 million (USD),1,4.2,1,0,0,1,1,1,other,junior,391,14.0,Remote
2067,I28 Technologies,Entry Level Data Science Specialist,"Marysville, MI",Must Have Knowledge of statistical techniques ...,57500,1 to 50 Employees,Company - Private,Financial Services,Accounting & Tax,NaN,Less than $1 million (USD),0,4.2,1,0,0,0,1,1,other,junior,73,NaN,MI
2080,Hunter Engineering,Data Science Co-Op,"Bridgeton, MO",Overview:\nDo you have a passion for data scie...,88383,1001 to 5000 Employees,Company - Private,Manufacturing,Machinery Manufacturing,1946.0,$500 million to $1 billion (USD),0,4.4,1,0,0,0,1,1,other,Senior,349,76.0,MO


**Let's clean the values that seems to fit in our groups:**

In [60]:
data.loc[data['company']=='I28 Technologies', 'job_simpl'] = data.loc[data['company']=='I28 Technologies', 'job_simpl'].replace('other', 'data scientist')
data.loc[data['company']=='Hunter Engineering', 'job_simpl'] = data.loc[data['company']=='Hunter Engineering', 'job_simpl'].replace('other', 'data scientist')
data.iloc[79, 19] = data.iloc[79, 19].replace('other', 'data scientist')
data.iloc[604, 19] = data.iloc[604, 19].replace('other', 'data scientist')
data.iloc[830, 19] = data.iloc[830, 19].replace('other', 'data scientist')

In [61]:
data.iloc[1915, 19] = data.iloc[1915, 19].replace('other', 'machine learning engineer')
data.iloc[1923, 19] = data.iloc[1923, 19].replace('other', 'machine learning engineer')

In [62]:
data.loc[data['company']=='NVIDIA', 'job_simpl'] = data.loc[data['company']=='NVIDIA', 'job_simpl'].replace('other', 'machine learning engineer')
data.loc[data['company']=='Bosch Group', 'job_simpl'] = data.loc[data['company']=='Bosch Group', 'job_simpl'].replace('other', 'machine learning engineer')
data.loc[data['company']=='Meta', 'job_simpl'] = data.loc[data['company']=='Meta', 'job_simpl'].replace('other', 'machine learning engineer')
data.loc[data['company']=='National Renewable Energy Laboratory', 'job_simpl'] = data.loc[data['company']=='National Renewable Energy Laboratory', 'job_simpl'].replace('other', 'machine learning engineer')

In [63]:
data.iloc[1882, 19] = data.iloc[1882, 19].replace('other', 'data engineer')
data.loc[data['company']=='Excelon Solutions', 'job_simpl'] = data.loc[data['company']=='Excelon Solutions', 'job_simpl'].replace('other', 'data engineer') 
data.loc[data['company']=='Sunixa Solution', 'job_simpl'] = data.loc[data['company']=='Sunixa Solution', 'job_simpl'].replace('other', 'data engineer')

In [64]:
data.loc[data['company']=='John Deere', 'job_simpl'] = data.loc[data['company']=='John Deere', 'job_simpl'].replace('other','data analyst') 
data.loc[data['company']=='NYC Careers', 'job_simpl'] = data.loc[data['company']=='NYC Careers', 'job_simpl'].replace('other','data analyst')
data.loc[data['company']=='First Notch Technolog', 'job_simpl'] = data.loc[data['company']=='First Notch Technolog', 'job_simpl'].replace('other','data analyst')
data.loc[data['company']=='B&H Photo', 'job_simpl'] = data.loc[data['company']=='B&H Photo', 'job_simpl'].replace('other','data analyst')

**Finally, let's drop the columns that are still considered as other**

In [65]:
data.drop(data[data['job_simpl']=='other'].index, inplace=True)

In [66]:
data['job_simpl'].value_counts(dropna=False)

data scientist               1231
data analyst                  333
data engineer                 313
machine learning engineer     180
Name: job_simpl, dtype: int64

In [67]:
data.isna().sum()

company                  0
job_title                0
location                 0
job_description          0
salary_estimate          0
company_size             0
company_type             0
company_sector           0
company_industry         0
company_founded        539
company_revenue          0
hourly                   0
rating                 277
python_yn                0
spark_yn                 0
azure_yn                 0
aws_yn                   0
excel_yn                 0
machine_learning_yn      0
job_simpl                0
seniority                0
description_len          0
company_age            539
state                    0
dtype: int64

In [68]:
def num_cat_splitter(df):
    df1 = df.copy()
    numerical_df = df1.select_dtypes(np.number)
    categorical_df = df1.select_dtypes(object)
    return numerical_df, categorical_df

In [69]:
numerical, categorical = num_cat_splitter(data)

**Now we fill NaN values of numericals with KNN imputer**

In [70]:
imputer = KNNImputer(n_neighbors=10)

numerical_np = imputer.fit_transform(numerical)
numerical_transformed = pd.DataFrame(numerical_np, columns=numerical.columns, index=numerical.index)

In [71]:
data_cleaned = pd.concat([categorical, numerical_transformed], axis=1)
data_cleaned

,company,job_title,location,job_description,company_size,company_type,company_sector,company_industry,company_revenue,job_simpl,seniority,state,salary_estimate,company_founded,hourly,rating,python_yn,spark_yn,azure_yn,aws_yn,excel_yn,machine_learning_yn,description_len,company_age
0,Microsoft,Data & Applied Scientist,"Redmond, WA",Microsoft 365 is a key part of the company’s c...,10000+ Employees,Company - Public,Information Technology,Computer Hardware Development,$10+ billion (USD),data scientist,junior,WA,123486.0,1975.0,0.0,4.40,1.0,0.0,0.0,1.0,0.0,1.0,359.0,47.0
1,UT Southwestern Medical Center,Data Scientist or Bioinformatician (remote),Remote,Center Information:\nThe Quantitative Biomedic...,10000+ Employees,Hospital,Healthcare,Health Care Services & Hospitals,$1 to $5 billion (USD),data scientist,mid,Remote,93500.0,1943.0,0.0,4.00,1.0,0.0,0.0,0.0,0.0,1.0,267.0,79.0
2,Notion,"Data Scientist, Growth","New York, NY",About Us:\nWe're on a mission to make it possi...,201 to 500 Employees,Company - Private,Information Technology,Enterprise Software & Network Solutions,Unknown / Non-Applicable,data scientist,Senior,NY,137853.0,2016.0,0.0,4.90,1.0,0.0,0.0,0.0,0.0,0.0,589.0,6.0
3,Net2Aspire,Jr. Data Scientist,Remote,? Apply Statistical and Machine Learning metho...,Unknown,Company - Public,Unknown,Unknown,Unknown / Non-Applicable,data scientist,junior,Remote,72500.0,1950.0,0.0,3.94,0.0,0.0,0.0,0.0,0.0,1.0,132.0,72.0
4,Ntropy Network,Data Scientist,Remote,"Over the last few decades, technological innov...",1 to 50 Employees,Company - Private,Unknown,Unknown,Unknown / Non-Applicable,data scientist,mid,Remote,155000.0,2011.0,0.0,4.07,1.0,0.0,0.0,1.0,0.0,0.0,522.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2078,Technical Paradigm,Python Backend Developer with Machine Learning.,Remote,We are looking for a Python backend developer ...,1 to 50 Employees,Private Practice / Firm,Unknown,Unknown,Unknown / Non-Applicable,machine learning engineer,Senior,Remote,100000.0,1977.0,1.0,5.00,1.0,0.0,0.0,1.0,0.0,1.0,269.0,45.0
2079,YouTube,"Staff Software Engineer, Machine Learning, You...","San Bruno, CA",Minimum qualifications:\nBachelor's degree or ...,1001 to 5000 Employees,Subsidiary or Business Segment,Information Technology,Internet & Web Services,Unknown / Non-Applicable,machine learning engineer,Senior,CA,141704.0,2005.0,0.0,4.30,0.0,0.0,0.0,0.0,0.0,1.0,498.0,17.0
2080,Hunter Engineering,Data Science Co-Op,"Bridgeton, MO",Overview:\nDo you have a passion for data scie...,1001 to 5000 Employees,Company - Private,Manufacturing,Machinery Manufacturing,$500 million to $1 billion (USD),data scientist,Senior,MO,88383.0,1946.0,0.0,4.40,1.0,0.0,0.0,0.0,1.0,1.0,349.0,76.0
2082,Argonne National Laboratory,Postdoctoral Appointee - Probabilistic Machine...,"Lemont, IL",The Mathematics and Computer Science Division ...,1001 to 5000 Employees,Government,Management & Consulting,Research & Development,Unknown / Non-Applicable,machine learning engineer,Senior,IL,54291.0,1946.0,0.0,4.50,0.0,0.0,0.0,0.0,0.0,1.0,506.0,76.0


In [72]:
data_cleaned.isna().sum()

company                0
job_title              0
location               0
job_description        0
company_size           0
company_type           0
company_sector         0
company_industry       0
company_revenue        0
job_simpl              0
seniority              0
state                  0
salary_estimate        0
company_founded        0
hourly                 0
rating                 0
python_yn              0
spark_yn               0
azure_yn               0
aws_yn                 0
excel_yn               0
machine_learning_yn    0
description_len        0
company_age            0
dtype: int64

In [73]:
data_cleaned.to_csv('../data/cleaned/data_cleaned.csv' , index=False)

## Now it's time to reduce the cardinality of the categorical columns

In [74]:
data_cleaned.columns

Index(['company', 'job_title', 'location', 'job_description', 'company_size',
       'company_type', 'company_sector', 'company_industry', 'company_revenue',
       'job_simpl', 'seniority', 'state', 'salary_estimate', 'company_founded',
       'hourly', 'rating', 'python_yn', 'spark_yn', 'azure_yn', 'aws_yn',
       'excel_yn', 'machine_learning_yn', 'description_len', 'company_age'],
      dtype='object')

In [75]:
columns_to_reduce_card = ['company_size','company_type', 'company_sector', 'company_industry', 'company_revenue', 'state']

for x in columns_to_reduce_card:
    print(data_cleaned[x].value_counts())

10000+ Employees           568
1 to 50 Employees          396
1001 to 5000 Employees     348
Unknown                    264
201 to 500 Employees       189
51 to 200 Employees        162
501 to 1000 Employees       74
5001 to 10000 Employees     56
Name: company_size, dtype: int64
Company - Private                 1001
Company - Public                   736
Subsidiary or Business Segment     108
Nonprofit Organization              54
Government                          50
College / University                41
Hospital                            28
Unknown                             16
Private Practice / Firm             10
Franchise                            7
Contract                             5
Self-employed                        1
Name: company_type, dtype: int64
Information Technology                         585
Unknown                                        357
Financial Services                             191
Management & Consulting                        141
Manufacturing 

In [76]:
data_cleaned_no_dup = data_cleaned.copy() #should I work without duplicates?? 

data_cleaned_no_dup.drop_duplicates(inplace=True)
data_cleaned_no_dup.shape

(747, 24)

In [77]:
data_cleaned_no_dup.to_csv('../data/cleaned/data_cleaned_no_duplicates.csv',index=False)

In [78]:
for x in columns_to_reduce_card:
    print(data_cleaned_no_dup[x].value_counts())

10000+ Employees           216
1001 to 5000 Employees     119
1 to 50 Employees          111
Unknown                     86
51 to 200 Employees         81
201 to 500 Employees        70
501 to 1000 Employees       36
5001 to 10000 Employees     28
Name: company_size, dtype: int64
Company - Private                 338
Company - Public                  272
Nonprofit Organization             35
Subsidiary or Business Segment     34
Government                         21
College / University               19
Unknown                             9
Hospital                            7
Private Practice / Firm             6
Franchise                           3
Contract                            2
Self-employed                       1
Name: company_type, dtype: int64
Information Technology                         188
Unknown                                        115
Management & Consulting                         60
Manufacturing                                   60
Financial Services        

### Grouping the column *company_size*

In [79]:
data_cleaned['company_size'].unique()

array(['10000+ Employees', '201 to 500 Employees', 'Unknown',
       '1 to 50 Employees', '1001 to 5000 Employees',
       '501 to 1000 Employees', '51 to 200 Employees',
       '5001 to 10000 Employees'], dtype=object)

In [80]:
def size_cleaner(x):
    small_company = ['1 to 50 Employees']
    medium_company = ['51 to 200 Employees','201 to 500 Employees']
    big_company = ['501 to 1000 Employees', '1001 to 5000 Employees', '5001 to 10000 Employees']
    huge_company = ['10000+ Employees']
    unknown_size = ['Unknown'] #we could assume that these companies are more likely to belong to small companies

    if x in small_company:
        return 'Small company'
    elif x in medium_company:
        return 'Medium-sized company'
    elif x in big_company:
        return 'Large company'
    elif x in huge_company:
        return 'Huge company'
    else:
        return 'Unknown size'

In [81]:
data_cleaned['size'] = list(map(size_cleaner, data_cleaned['company_size']))
data_cleaned['size'].value_counts(dropna=False)

Huge company            568
Large company           478
Small company           396
Medium-sized company    351
Unknown size            264
Name: size, dtype: int64

### Grouping the column *company_type*

In [82]:
data_cleaned['company_type'].unique()

array(['Company - Public', 'Hospital', 'Company - Private',
       'Subsidiary or Business Segment', 'Unknown', 'Government',
       'Private Practice / Firm', 'College / University',
       'Nonprofit Organization', 'Franchise', 'Contract', 'Self-employed'],
      dtype=object)

In [83]:
def type_cleaner(x):
    other = ['Hospital', 'Subsidiary or Business Segment', 'Unknown', 'Government', 'Private Practice / Firm', 'College / University',
             'Nonprofit Organization', 'Franchise', 'Contract', 'Self-employed']

    if x in other:
        return 'Other institutions'
    else:
        return x

In [84]:
data_cleaned['type'] = list(map(type_cleaner, data_cleaned['company_type']))
data_cleaned['type'].value_counts(dropna=False)

Company - Private     1001
Company - Public       736
Other institutions     320
Name: type, dtype: int64

### Grouping the column *company_sector*

In [85]:
data_cleaned['company_sector'].value_counts()

Information Technology                         585
Unknown                                        357
Financial Services                             191
Management & Consulting                        141
Manufacturing                                  120
Healthcare                                     118
Media & Communication                           79
Pharmaceutical & Biotechnology                  74
Human Resources & Staffing                      67
Education                                       50
Retail & Wholesale                              41
Government & Public Administration              38
Transportation & Logistics                      33
Aerospace & Defense                             29
Real Estate                                     26
Insurance                                       19
Energy, Mining & Utilities                      17
Restaurants & Food Service                      17
Telecommunications                              15
Nonprofit & NGO                

In [86]:
data_cleaned['company_sector'].value_counts().index

Index(['Information Technology', 'Unknown', 'Financial Services',
       'Management & Consulting', 'Manufacturing', 'Healthcare',
       'Media & Communication', 'Pharmaceutical & Biotechnology',
       'Human Resources & Staffing', 'Education', 'Retail & Wholesale',
       'Government & Public Administration', 'Transportation & Logistics',
       'Aerospace & Defense', 'Real Estate', 'Insurance',
       'Energy, Mining & Utilities', 'Restaurants & Food Service',
       'Telecommunications', 'Nonprofit & NGO',
       'Construction, Repair & Maintenance Services',
       'Arts, Entertainment & Recreation', 'Hotels & Travel Accommodation',
       'Agriculture', 'Legal'],
      dtype='object')

In [87]:
def sector_cleaner(x):
    technology = ['Information Technology', 'Telecommunications']
    business_services = ['Management & Consulting', 'Human Resources & Staffing']
    financial_services_and_housing =['Financial Services', 'Real Estate', 'Insurance']
    media_and_entertaiment = ['Media & Communication', 'Arts, Entertainment & Recreation']
    industry = ['Manufacturing', 'Energy, Mining & Utilities', 'Construction, Repair & Maintenance Services']
    health = ['Healthcare', 'Pharmaceutical & Biotechnology']
    other = ['Education', 'Retail & Wholesale','Government & Public Administration', 'Transportation & Logistics','Aerospace & Defense', 'Restaurants & Food Service', 
             'Nonprofit & NGO','Hotels & Travel Accommodation','Agriculture', 'Legal'] 

    if x in technology:
        return 'IT and Telecommunications'
    elif x in business_services:
        return 'Business services'
    elif x in financial_services_and_housing:
        return 'Financial and housing services'
    elif x in media_and_entertaiment:
        return 'Media and Entertaiment'
    elif x in industry:
        return 'Industry'
    elif x in health:
        return 'Healthcare'
    elif x in other:
        return 'Other sectors'
    else:
        return x

In [88]:
data_cleaned['sector'] = list(map(sector_cleaner, data_cleaned['company_sector']))
data_cleaned['sector'].value_counts(dropna=False)

IT and Telecommunications         600
Unknown                           357
Financial and housing services    236
Other sectors                     232
Business services                 208
Healthcare                        192
Industry                          145
Media and Entertaiment             87
Name: sector, dtype: int64

**I will not clean the column company_industry because this step will be redundant with the sector and will not add much more info**

## Grouping the column *company_revenue*

In [89]:
data_cleaned['company_revenue'].value_counts()

Unknown / Non-Applicable            809
$10+ billion (USD)                  389
$1 to $5 billion (USD)              247
$5 to $25 million (USD)             126
Less than $1 million (USD)          121
$1 to $5 million (USD)               95
$25 to $100 million (USD)            87
$100 to $500 million (USD)           74
$5 to $10 billion (USD)              68
$500 million to $1 billion (USD)     41
Name: company_revenue, dtype: int64

In [90]:
def revenue_cleaner(x):
    less_than_5 = ['Less than $1 million (USD)', '$1 to $5 million (USD)']
    two_figure_millionare = ['$5 to $25 million (USD)','$25 to $100 million (USD)']
    three_figure_millionare = ['$100 to $500 million (USD)', '$500 million to $1 billion (USD)']
    billionare = ['$1 to $5 billion (USD)', '$5 to $10 billion (USD)']
    two_figure_billionare = ['$10+ billion (USD)']

    if x in less_than_5:
        return 'Less than $5 million (USD)'
    elif x in two_figure_millionare:
        return '$5 to $100 million (USD)'
    elif x in three_figure_millionare:
        return '$100 million to $1 billion (USD)'
    elif x in billionare:
        return '$1 to $10 billion (USD)'
    else:
        return x

In [91]:
data_cleaned['revenue'] = list(map(revenue_cleaner, data_cleaned['company_revenue']))
data_cleaned['revenue'].value_counts(dropna=False)

Unknown / Non-Applicable            809
$10+ billion (USD)                  389
$1 to $10 billion (USD)             315
Less than $5 million (USD)          216
$5 to $100 million (USD)            213
$100 million to $1 billion (USD)    115
Name: revenue, dtype: int64

## Grouping the column *state*

In [92]:
data_cleaned['state'].value_counts(dropna=False)

Remote    557
CA        266
NY        230
IL        111
MA         90
TX         77
WA         68
VA         66
MI         62
PA         58
CO         48
MD         45
NC         41
UT         38
OH         38
NJ         37
SC         29
MO         28
FL         27
GA         27
NM         19
AZ         10
MN         10
DC         10
WI          7
CT          7
DE          6
OK          6
LA          6
ID          5
IA          5
IN          4
AR          3
NV          3
TN          3
OR          3
AL          1
AF          1
MS          1
RI          1
KS          1
NE          1
PR          1
Name: state, dtype: int64

In [93]:
def state_cleaner(x):
    northeastern = ['CT','ME','MA','NH','NJ','NY','PA','RI','VT']
    midwestern = ['IL','IN','IA','KS','MI','MN','MO','NE','ND','OH','SD','WI']
    southern = ['AL','AR','DE','FL','GA','KY','LA','MD','MS','NC','OK','SC','TN','TX','VA','WV','DC','AF','PR']
    western = ['AK','AZ','CA','CO','HI','ID','MT','NV','NM','OR','UT','WA','WY']

    if x in northeastern:
        return 'Northeastern'
    elif x in midwestern:
        return 'Midwestern'
    elif x in southern:
        return 'Southern'
    elif x in western:
        return 'Western'
    else:
        return x

In [94]:
data_cleaned['zone'] = list(map(state_cleaner, data_cleaned['state']))
data_cleaned['zone'].value_counts(dropna=False)

Remote          557
Western         460
Northeastern    423
Southern        350
Midwestern      267
Name: zone, dtype: int64

In [95]:
data_cleaned.to_csv('../data/cleaned/data_cleaned_and_grouped.csv',index=False)